In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Cell 1: Installations
!pip install -q timm py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.6 MB/s eta 0:00

In [12]:
# Cell 2: Imports & Configuration
import os
import pandas as pd
import numpy as np
from PIL import Image
import py7zr
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import timm

# --- Configuration ---
class CFG:
    # Kaggle environment paths
    data_path = '/kaggle/input/cifar10-python/cifar-10-batches-py'
    train_archive = os.path.join(data_path, 'train.npy')
    test_archive = os.path.join(data_path, 'test.npy')
    train_labels_csv = os.path.join(data_path, 'trainLabels.csv')
    
    # Output paths
    extract_path = '/kaggle/working/cifar10/'
    train_img_path = os.path.join(extract_path, 'train')
    test_img_path = os.path.join(extract_path, 'test')
    model_save_path = '/kaggle/input/iisc-vit/pytorch/default/1/iisc vit.pth'

    # Model and Training parameters
    model_name = 'vit_base_patch16_224.augreg_in21k' # A powerful pre-trained ViT
    img_size = 224
    batch_size = 32
    epochs = 50  # A good starting point for fine-tuning
    learning_rate = 1e-4
    weight_decay = 1e-5
    label_smoothing = 0.1
    validation_split = 0.1 # Use 10% of training data for validation
    
    # Hardware
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # CIFAR-10 specific
    num_classes = 10
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Using device: {CFG.device}")

Using device: cuda


In [4]:
import pickle
import numpy as np

def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
    data = batch['data']
    labels = batch['labels']
    data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (N, 32, 32, 3)
    return data, np.array(labels)  # <-- convert to numpy here

# Load all training batches
train_data, train_labels = [], []
for i in range(1, 6):
    data, labels = load_cifar_batch(os.path.join(CFG.data_path, f'data_batch_{i}'))
    train_data.append(data)
    train_labels.append(labels)

train_data = np.concatenate(train_data)         # (50000, 32, 32, 3)
train_labels = np.concatenate(train_labels)     # (50000,)

# Load test batch
test_data, test_labels = load_cifar_batch(os.path.join(CFG.data_path, 'test_batch'))

print("Train data:", train_data.shape, "Labels:", train_labels.shape)
print("Test data:", test_data.shape, "Labels:", test_labels.shape)


Train data: (50000, 32, 32, 3) Labels: (50000,)
Test data: (10000, 32, 32, 3) Labels: (10000,)


In [5]:
os.makedirs(CFG.train_img_path, exist_ok=True)
os.makedirs(CFG.test_img_path, exist_ok=True)

# Save training images
for idx, (img_array, label) in tqdm(enumerate(zip(train_data, train_labels)), total=len(train_data)):
    class_name = CFG.class_names[label]
    class_dir = os.path.join(CFG.train_img_path, class_name)
    os.makedirs(class_dir, exist_ok=True)
    img = Image.fromarray(img_array.astype('uint8'))
    img.save(os.path.join(class_dir, f"{idx}.png"))

# Save test images
for idx, (img_array, label) in tqdm(enumerate(zip(test_data, test_labels)), total=len(test_data)):
    class_name = CFG.class_names[label]
    class_dir = os.path.join(CFG.test_img_path, class_name)
    os.makedirs(class_dir, exist_ok=True)
    img = Image.fromarray(img_array.astype('uint8'))
    img.save(os.path.join(class_dir, f"{idx}.png"))


  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
print("Expected train samples:", train_data.shape[0])
print("Files saved:", len(os.listdir(CFG.train_img_path)))


Expected train samples: 50000
Files saved: 10


In [9]:
# Cell 4: Dataset & DataLoaders

class Cifar10Dataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_array = self.images[idx]
        image = Image.fromarray(img_array.astype('uint8'))  # Convert numpy -> PIL

        if self.transform:
            image = self.transform(image)

        if self.labels is not None:  # training/validation
            label = self.labels[idx]
            return image, torch.tensor(label, dtype=torch.long)
        else:  # test set (labels unknown)
            return image, idx

# Define data augmentations and normalization
data_transforms = {
    'train': T.Compose([
        T.Resize((CFG.img_size, CFG.img_size)),
        T.TrivialAugmentWide(),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ]),
    'val': T.Compose([
        T.Resize((CFG.img_size, CFG.img_size)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
}

# --- Create full dataset from numpy arrays ---
full_dataset = Cifar10Dataset(train_data, train_labels, transform=None)

# Split into train/validation
val_size = int(len(full_dataset) * CFG.validation_split)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply transforms to each split
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# --- Create DataLoaders ---
train_loader = DataLoader(
    train_dataset, batch_size=CFG.batch_size, shuffle=True,
    num_workers=os.cpu_count(), pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=CFG.batch_size, shuffle=False,
    num_workers=os.cpu_count(), pin_memory=True
)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


Training samples: 45000, Validation samples: 5000


In [10]:
# Cell 5: Model, Loss, Optimizer Setup

# Load pre-trained ViT model
model = timm.create_model(
    CFG.model_name,
    pretrained=True,
    num_classes=CFG.num_classes
)
model.to(CFG.device)

# Loss Function with Label Smoothing
# This regularization prevents the model from becoming overconfident
loss_fn = nn.CrossEntropyLoss(label_smoothing=CFG.label_smoothing)

# AdamW Optimizer - better than Adam for generalization
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.learning_rate, weight_decay=CFG.weight_decay)

# OneCycleLR Scheduler for faster convergence and better performance
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=CFG.learning_rate,
    steps_per_epoch=len(train_loader),
    epochs=CFG.epochs
)

# Gradient Scaler for Automatic Mixed Precision (AMP)
# This speeds up training significantly on modern GPUs
scaler = torch.cuda.amp.GradScaler()

model.safetensors:   0%|          | 0.00/410M [00:00<?, ?B/s]

/tmp/ipykernel_36/3930098787.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
# Cell 6: Training & Validation Loop

def train_one_epoch(model, loader, optimizer, scheduler, loss_fn, scaler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(loader, desc="Training", leave=False)
    
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Mixed precision training context
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
        
        # Backward pass with scaler
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        scheduler.step() # Step scheduler at each batch for OneCycleLR
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item(), lr=scheduler.get_last_lr()[0])
        
    return total_loss / len(loader)

def validate_one_epoch(model, loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        progress_bar = tqdm(loader, desc="Validating", leave=False)
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            total_samples += labels.size(0)

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions.double() / total_samples
    return avg_loss, accuracy

# --- Main Training Loop ---
best_val_accuracy = 0.0

for epoch in range(CFG.epochs):
    print(f"--- Epoch {epoch+1}/{CFG.epochs} ---")
    
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, loss_fn, scaler, CFG.device)
    val_loss, val_accuracy = validate_one_epoch(model, val_loader, loss_fn, CFG.device)
    
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"New best model found! Saving to {CFG.model_save_path}")
        torch.save(model.state_dict(), CFG.model_save_path)

print("\n--- Training Finished ---")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")


--- Epoch 1/50 ---


Training:   0%|          | 0/1407 [00:00<?, ?it/s]

/tmp/ipykernel_36/2259735591.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Validating:   0%|          | 0/157 [00:00<?, ?it/s]

/tmp/ipykernel_36/2259735591.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


...

Epoch 25: Train Loss: 0.5269 | Val Loss: 0.6418 | Val Accuracy: 0.9438
--- Epoch 26/50 ---


Training:   0%|          | 0/1407 [00:00<?, ?it/s]

Validating:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 26: Train Loss: 0.5208 | Val Loss: 0.6291 | Val Accuracy: 0.9506
--- Epoch 27/50 ---


Training:   0%|          | 0/1407 [00:00<?, ?it/s]

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import pickle

# 1. Reload the test batch
def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
    data = batch['data']
    labels = batch['labels']
    data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (N, 32, 32, 3)
    return data, np.array(labels)

test_data, test_labels = load_cifar_batch("/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch")

# 2. Create Dataset class (same as training)
class Cifar10Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_array = self.images[idx]
        image = Image.fromarray(img_array.astype('uint8'))  # numpy -> PIL
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, torch.tensor(label, dtype=torch.long)
        else:
            return image, idx

# 3. Use the same validation transforms
test_transform = T.Compose([
    T.Resize((CFG.img_size, CFG.img_size)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_dataset = Cifar10Dataset(test_data, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=os.cpu_count())

# 4. Load the trained model
model = timm.create_model(CFG.model_name, pretrained=False, num_classes=CFG.num_classes)
model.load_state_dict(torch.load(CFG.model_save_path, map_location=CFG.device))
model.to(CFG.device)
model.eval()

# 5. Evaluate accuracy
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(CFG.device), labels.to(CFG.device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total * 100
print(f"✅ Test Accuracy: {test_accuracy:.2f}%")
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import pickle

# 1. Reload the test batch
def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
    data = batch['data']
    labels = batch['labels']
    data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (N, 32, 32, 3)
    return data, np.array(labels)

test_data, test_labels = load_cifar_batch("/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch")

# 2. Create Dataset class (same as training)
class Cifar10Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_array = self.images[idx]
        image = Image.fromarray(img_array.astype('uint8'))  # numpy -> PIL
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, torch.tensor(label, dtype=torch.long)
        else:
            return image, idx

# 3. Use the same validation transforms
test_transform = T.Compose([
    T.Resize((CFG.img_size, CFG.img_size)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_dataset = Cifar10Dataset(test_data, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=os.cpu_count())

# 4. Load the trained model
model = timm.create_model(CFG.model_name, pretrained=False, num_classes=CFG.num_classes)
model.load_state_dict(torch.load(CFG.model_save_path, map_location=CFG.device))
model.to(CFG.device)
model.eval()

# 5. Evaluate accuracy
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(CFG.device), labels.to(CFG.device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total * 100
print(f"✅ Test Accuracy: {test_accuracy:.2f}%")


✅ Test Accuracy: 98.77%
✅ Test Accuracy: 98.77%
